In [106]:
import pandas as pd
import os
import numpy as np
import pytz
from datetime import datetime
from sort_resample_organize import resample
import warnings
warnings.filterwarnings("ignore")

### Content
<ol>
    <li>Merge raw accelerometer and gyroscope data</li>
    <li>Include only the in-lab session</li>
    <li>Resampling to 20 hz</li>
</ol>

In [107]:
ROOT_PATH_ACC = '/Wild/Wrist/Clean/Original/Accelerometer/'
ROOT_PATH_GYRO = '/Wild/Wrist/Clean/Original/Gyroscope/'
ROOT_PATH_FSM = 'Y:/PrevMed/Alshurafa_Lab/Lab_Common/CalorieHarmony/A. Phase 2 Participants/P1000/'
PATH_RESAMPLE_ACC = '/Wild/Wrist/Clean/Resampled/Accelerometer/'
PATH_RESAMPLE_GYRO = '/Wild/Wrist/Clean/Resampled/Gyroscope/'

In [51]:
participant_list = ['1000','1001','1002','1003','1004','1005','1006','1007','1008','1009','1010','1011','1012','1013','1014','1015']
met_cart_dic = {'1000':'01/27/2022 13:43:15',
                '1001':'NA',
                '1002':'02/09/2022 10:30:45',
                '1003':'01/12/2022 15:47:36',
                '1004':'01/21/2022 13:49:59',
                '1005':'02/12/2022 11:57:42',
                '1006':'03/22/2022 10:23:05',
                '1007':'03/28/2022 12:49:21',
                '1008':'04/12/2022 12:11:31',
                '1009':'04/15/2022 12:37:46',
                '1010':'05/06/2022 11:14:05',
                '1011':'05/11/2022 12:02:56',
                '1012':'05/24/2022 16:42:47',
                '1013':'05/27/2022 11:34:26',
                '1014':'06/03/2022 11:50:35',
                '1015':'06/06/2022 10:41:15'
}

In [65]:
def merge_files(path):
    dir_list = os.listdir(path)
    dir_list.sort()
    df_list = []
    for each_dir in dir_list:
        df_list.append(pd.read_csv(path + each_dir))
    df_merged = pd.concat(df_list)    
    df_merged['date'] = pd.to_datetime(df_merged['datetime']).dt.date
    df_merged = df_merged.sort_values(by=['datetime'],ascending=True).reset_index(drop=True)
    return(df_merged)

def get_utc_date(local_time_string):
    # local_time_string: met_cart_dic['1000']
    local = pytz.timezone("America/Chicago")
    naive = datetime.strptime(local_time_string, '%m/%d/%Y %H:%M:%S')
    local_dt = local.localize(naive, is_dst=None)
    utc_dt = local_dt.astimezone(pytz.utc)
    return(utc_dt.strftime("%Y-%m-%d"))

In [135]:
for p in participant_list:
    print('processing: ', p)
    try:
        df_acc = merge_files('data_phase_2/' + str(p) + ROOT_PATH_ACC)
        df_gyro = merge_files('data_phase_2/' + str(p) + ROOT_PATH_GYRO)

        df_acc_corrected = df_acc[df_acc['date']==pd.to_datetime(get_utc_date(met_cart_dic[str(p)]))].reset_index(drop=True)
        df_gyro_corrected = df_gyro[df_gyro['date']==pd.to_datetime(get_utc_date(met_cart_dic[str(p)]))].reset_index(drop=True)

        # resample
        df_resample_acc = resample(df_acc_corrected[['Time','accX','accY','accZ']], 'Time', 20)
        df_resample_gyro = resample(df_gyro_corrected[['Time','rotX', 'rotY', 'rotZosboxe']], 'Time', 20)

        df_resample_acc.to_csv(os.path.join('data_phase_2/'+str(p)+PATH_RESAMPLE_ACC, 'acc_resample.csv'),index=False)
        df_resample_gyro.to_csv(os.path.join('data_phase_2/'+str(p)+PATH_RESAMPLE_GYRO, 'gyro_resample.csv'),index=False)
    except:
        print('Invalid:', p)

processing:  1000
processing:  1001
Invalid: 1001
processing:  1002
processing:  1003
processing:  1004
processing:  1005
processing:  1006
processing:  1007
processing:  1008
processing:  1009
processing:  1010
processing:  1011
processing:  1012
processing:  1013
processing:  1014
processing:  1015
